## Molecular Dynamics

Run molecular dynamics (MD) simulations using [GPUMD](https://github.com/brucefan1983/GPUMD).

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Creating bulk atoms.

In [ ]:
from wizard.atoms import SymbolInfo, Morph
atoms = SymbolInfo('MoTaVW', 'bcc', 3).create_bulk_atoms((20, 20, 20))

Running molecular dynamics simulations.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 300', 
          'time_step 1', 
          'ensemble npt_scr 300 300 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_restart 30000', 
          'dump_exyz 10000',
          'run 30000']
Morph(atoms).gpumd('relax', run_in)

Deforming the simulation box.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 300', 
          'time_step 1',
          'ensemble npt_scr 300 300 100 0 0 0 100 100 100 1000',
          'run 30000', 
          'ensemble npt_scr 300 300 100 0 0 0 100 100 100 1000',
          'deform 0.00001 0 0 1', 
          'dump_thermo 1000', 
          'dump_exyz 1000', 
          'dump_restart 10000',
          'run 1000000']
Morph(atoms).gpumd('deform', run_in)

Simulating the process of a crystallization.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 2000', 
          'time_step 1', 
          'ensemble npt_scr 2000 2000 200 0 500 2000', 
          'dump_thermo 1000', 
          'dump_exyz 10000', 
          'dump_restart 10000', 
          'run 100000',
          'ensemble npt_scr 2000 5000 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_exyz 100000', 
          'dump_restart 10000', 
          'run 10000000',
          'ensemble npt_scr 4500 4500 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_exyz 10000', 
          'dump_restart 10000', 
          'run 100000',
          'ensemble npt_scr 4500 1500 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_exyz 100000', 
          'dump_restart 10000', 
          'run 10000000']
Morph(atoms).gpumd('crystallization', run_in)

Calculating the Melting point using two-phase coexistence method.

In [ ]:
from wizard.io import read_restart

group = []
for atom in atoms:
    if atom.position[2] < atoms.cell[2, 2] / 2:
        group.append(0)
    else:
        group.append(1)
atoms.info['group'] = group

run_in_1 = ['potential ../../nep.txt', 
            'velocity 3000', 
            'time_step 1', 
            'ensemble npt_ber 3000 3000 200 0 500 2000', 
            'dump_exyz 10000', 
            'dump_thermo 1000',
            'run 30000',
            'ensemble heat_lan 3500 200 500 0 1',
            'dump_exyz 10000',
            'dump_thermo 1000',
            'dump_restart 10000',
            'run 1000000']

Morph(atoms).gpumd('melting_point/relax', run_in_1)

for Tm in range(3400, 3701, 100):
    atoms = read_restart('melting_point/relax/restart.xyz')
    run_in = ['potential ../../nep.txt', 
             f'velocity {Tm}', 
              'time_step 1', 
             f'ensemble npt_ber {Tm} {Tm} 200 0 500 2000', 
              'dump_exyz 10000', 
              'dump_thermo 1000',
              'run 30000']
    Morph(atoms).gpumd(f'melting_point/{Tm}', run_in)

Simulating the radiation damage.

In [ ]:
from wizard.io import read_restart
import numpy as np

group = []
thickness = 3.185 * 3
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
atoms.info['group'] = group

run_in_1 = ['potential nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 30000', 
            'run 30000']

run_in_2 = ['potential nep.txt', 
            'velocity 300', 
            'time_step 0', 
            'ensemble nve',
            'dump_exyz 1', 
            'run 1',
            'time_step 1 0.015', 
            'ensemble heat_nhc 300 200 0 0 1',
            'compute 0 200 10 temperature', 
            'dump_restart 10000', 
            'dump_exyz 2000 1 1',
            'run 70000']

pka_energy = 10 #eV
direction = np.array([1, 3, 5]) 

Morph(atoms).gpumd('radiation/relax', run_in_1)
atoms = read_restart('radiation/relax/restart.xyz')
Morph(atoms).set_pka(pka_energy, direction)
Morph(atoms).gpumd('radiation/cascade', run_in_2)

Simulating the overlapping cascades.

In [ ]:
from wizard.io import read_restart
import numpy as np
import random

run_in_1 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 10000', 
            'run 30000']

run_in_2 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 10000', 
            'run 10000']

run_in_3 = ['potential ../../nep.txt', 
            'velocity 300', 
            'time_step 0', 
            'ensemble nve',
            'dump_exyz 1', 
            'run 1',
            'time_step 1 0.015', 
            'ensemble heat_nhc 300 200 0 0 1',
            'electron_stop ../../electron_stopping_fit.txt',
            'compute 0 200 10 temperature', 
            'dump_restart 10000', 
            'dump_exyz 2000 1 1',
            'run 40000']

pka_energy = 10 #eV
cascade_times = 2000
directions = [np.array([np.sin(np.random.uniform(0, np.pi)) * np.cos(np.random.uniform(0, 2 * np.pi)),
                        np.sin(np.random.uniform(0, np.pi)) * np.sin(np.random.uniform(0, 2 * np.pi)),
                        np.cos(np.random.uniform(0, np.pi))]) for _ in range(cascade_times)]

indexs = [random.randint(0, len(atoms) - 1) for _ in range(cascade_times)]

## First time
direction = directions[0]
index = indexs[0]

center = atoms.cell.diagonal() / 2
diff = center - atoms[index].position
for atom in atoms:
    atom.position += diff

for atom in atoms:
    atom.position %= atoms.cell.diagonal()

group = []
thickness = 3.185 * 3
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
atoms.info['group'] = group

Morph(atoms).gpumd('radiation0/relax', run_in_1)
atoms = read_restart('radiation0/relax/restart.xyz')
Morph(atoms).set_pka(pka_energy, direction, index)
Morph(atoms).gpumd('radiation0/cascade', run_in_3)

## Loops
for i in range(1, cascade_times):
    direction = directions[i]
    index = indexs[i]
    atoms = read_restart(f'radiation{i-1}/cascade/restart.xyz')

    center = atoms.cell.diagonal() / 2
    diff = center - atoms[index].position
    for atom in atoms:
        atom.position += diff

    for atom in atoms:
        atom.position %= atoms.cell.diagonal()

    group = []
    thickness = 3.185 * 3
    for atom in atoms:
        if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
            group.append(0)
        elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
            group.append(1)
        else:
            group.append(2)
    atoms.info['group'] = group

    Morph(atoms).gpumd(f'radiation{i}/relax', run_in_2)
    atoms = read_restart(f'radiation{i}/relax/restart.xyz')
    Morph(atoms).set_pka(pka_energy, direction, index)
    Morph(atoms).gpumd(f'radiation{i}/cascade', run_in_3)

Calculate the threshold displacement energy surface

In [ ]:
from wizard.atoms import SymbolInfo, Morph
from wizard.io import read_restart
import numpy as np

atoms = SymbolInfo('W', 'bcc', 3.185).create_bulk_atoms(supercell=(12,12,16))
group = []
thickness = 3.185 * 1
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
atoms.info['group'] = group

run_in_1 = ['potential nep.txt',
            'velocity 36', 
            'time_step 1', 
            'ensemble nvt_mttk temp 36 36', 
            'dump_restart 1', 
            'run 1']

run_in_2 = ['potential nep.txt', 
            'velocity 300', 
            'time_step 0', 
            'ensemble nve',
            'dump_exyz 1', 
            'run 1',
            'time_step 1 0.015', 
            'ensemble heat_nhc 36 200 0 0 1',
            'dump_exyz 100 1 1',
            'run 5000']

Morph(atoms).gpumd('TDE/initial', run_in_1)
atoms = read_restart('TDE/initial/restart.xyz')

directions = []
r = 1
for sita in np.linspace(0, np.pi / 4, 10):
    for phi in np.linspace(0, np.pi / 4 , 10):
        x = r * np.cos(phi) * np.cos(sita)
        y = r * np.cos(phi) * np.sin(sita)
        z = r * np.sin(phi)
        directions.append(np.array([x, y, z]))

for direction in directions:
    for pka_energy in range(40, 200, 20):
        Morph(atoms).set_pka(pka_energy, direction)
        direction_path = ''.join([str(i) for i in direction])
        Morph(atoms).gpumd(f'TDE/{direction_path}/{pka_energy}', run_in_2)

Calculate the formation eneregy of interstitial clusters.

In [ ]:
from wizard.atoms import SymbolInfo, Morph
from wizard.io import read_xyz
import numpy as np
       
burger = (1, 0, 0)
Rcut = 10
thickness = 2
atoms = SymbolInfo('W', 'bcc', 3.185).create_bulk_atoms(supercell=(30, 30, 30))
atoms_energy = 12.597
center = atoms.get_center_of_mass()
for atom in atoms:
    vector = atom.position - center
    proj = abs(vector @ burger) / np.linalg.norm(burger)
    R = np.sqrt(max(np.dot(vector, vector) - np.dot(proj, proj), 0))
    if  R < Rcut and proj < thickness:
        Morph(atoms).create_self_interstitial_atom(burger, index = atom.index)
Morph(atoms).gpumd('sia_cluster',['potential nep.txt', 'ensemble nve', 'time_step 0',
                                  'minimize fire 1.0e-4 1000','dump_exyz 1','run 1'])
frames = read_xyz('sia_cluster/dump.xyz')
atoms = frames[-1]
formation_energy = atoms.info['energy'] -  atoms_energy * len(atoms)

Simulating by ASE

In [ ]:
from wizard.atoms import SymbolInfo
from wizard.molecular_dynamics import MolecularDynamics
from pynep.calculate import NEP

calc = NEP('../Repository/UNEP_v1/nep.txt')
atoms = SymbolInfo('Al', 'fcc', 3).create_bulk_atoms()
MolecularDynamics(atoms, calc).NPT(steps=1000)